In [9]:
import pandas as pd
import numpy as np
from scipy.special import rel_entr, entr, xlogy

In [10]:
# List of available dataset
datasets = ["DTD","EuroSAT","GTSRB","MNIST","SVHN","Caltech256","SUN397","iWildCam","iWildCamID","CINIC10","CIFAR10"]

In [11]:
lb = 1e-5
ub = 100
lap_eps = 1e-5
def laplace_pred_mat(P):
    P+=lap_eps
    row_sums = P.sum(axis=1)
    normalized_P = P.div(row_sums, axis=0)
    return normalized_P
    
def calc_kl(P, Q):
    P = laplace_pred_mat(P)
    Q = laplace_pred_mat(Q)
    return np.sum(rel_entr(P, Q).clip(lb,ub), axis=1).clip(lb,ub)

# Calculate Entropy
def calc_entropy(P):
    P = laplace_pred_mat(P)
    return np.sum(entr(P), axis=1).clip(lb,ub)

# Calculate Cross-Entropy
def calc_cross_ent(P, Q):
    P = laplace_pred_mat(P)
    Q = laplace_pred_mat(Q)
    return (-np.sum(xlogy(P,Q), axis=1)).clip(lb,ub)

def calc_nll(p):
    P = laplace_pred_mat(P)
    return -np.log(p)


In [12]:


for ds in datasets:
    df18 = pd.read_csv(f"../Pred/{ds}_Resnet18.csv")
    df50 = pd.read_csv(f"../Pred/{ds}_Resnet50.csv")
    df101 = pd.read_csv(f"../Pred/{ds}_Resnet101.csv")
    label = pd.read_csv(f"../Pred/target_{ds}.csv")
    prediction = pd.DataFrame({
    "pred18": df18.idxmax(axis=1),
    "pred50": df50.idxmax(axis=1),
    "pred101": df101.idxmax(axis=1),
    "target": label['target']
    })
    prediction["pred18"] = prediction["pred18"].str.extract('(\d+)').astype(int)
    prediction["pred50"] = prediction["pred50"].str.extract('(\d+)').astype(int)
    prediction["pred101"] = prediction["pred101"].str.extract('(\d+)').astype(int)
    prediction.to_csv(f"csv/prediction_{ds}.csv")
    unc_pred = pd.DataFrame()
    unc_pred["kl_18_101"] = calc_kl(df18,df101)
    unc_pred["kl_101_18"] = calc_kl(df101,df18)
    unc_pred["kl_18_50"] = calc_kl(df18,df50)
    unc_pred["kl_50_18"] = calc_kl(df50,df18)
    unc_pred["ent_18"] = calc_entropy(df18)
    unc_pred["ent_50"] = calc_entropy(df50)
    unc_pred["ent_101"] = calc_entropy(df101)
    unc_pred["ce_18_101"] = calc_cross_ent(df18,df101)
    unc_pred["ce_101_18"] = calc_cross_ent(df101,df18)
    unc_pred["ce_18_50"] = calc_cross_ent(df18,df50)
    unc_pred["ce_50_18"] = calc_cross_ent(df50,df18)
    # Calculate NLL and Brier Scores for each sample and add as columns
    # ==============
    flat_label = label['target'].values.flatten()
    df18_array = df18.values
    df50_array = df50.values
    df101_array = df101.values
    # print("df18_array shape:", df18_array.shape)
    # print("df101_array shape:", df101_array.shape)
    # print("flat_label shape:", flat_label.shape)
    # Extracting the predicted probabilities for the true class labels
    predicted_probabilities18 = df18_array[np.arange(df18_array.shape[0]), flat_label]
    predicted_probabilities50 = df50_array[np.arange(df50_array.shape[0]), flat_label]
    predicted_probabilities101 = df101_array[np.arange(df101_array.shape[0]), flat_label]
    # NLL
    unc_pred["nll_18"] = (-np.log(predicted_probabilities18)).clip(0)
    unc_pred["nll_50"] = (-np.log(predicted_probabilities50)).clip(0)
    unc_pred["nll_101"] = (-np.log(predicted_probabilities101)).clip(0)
    # Brier Scores:
    unc_pred["brier_18"] = (predicted_probabilities18-1)**2 + np.sum(np.square(df18),axis=1) - predicted_probabilities18**2
    unc_pred["brier_50"] = (predicted_probabilities50-1)**2 + np.sum(np.square(df50),axis=1) - predicted_probabilities50**2
    unc_pred["brier_101"] = (predicted_probabilities101-1)**2 + np.sum(np.square(df101),axis=1) - predicted_probabilities101**2
    
    unc_pred["softmax_response18"] = np.max(df18_array,axis=1)
    unc_pred["softmax_response50"] = np.max(df50_array,axis=1)
    unc_pred["softmax_response101"] = np.max(df101_array,axis=1)
    
    unc_pred.to_csv(f"csv/uncertainty_{ds}.csv")